In [ ]:
<a href="https://colab.research.google.com/github/rgclapp007/gp211-class-notebooks/blob/main/preconditioning/seabin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install "sep_plot @ git+http://zapad.stanford.edu/bob/pySepPlot.git@2bffacb9fb36963339a0834c2b04a0aedff91db4"


O(m)=||d-Fm||+e^2||Am|

O(p) || F P p||



In [ ]:
!wget https://raw.githubusercontent.com/rgclapp007/gp211-class-notebooks/main/data/seabin.HH

In [ ]:
import sep_python.modes
import numpy as np
from sep_python.hypercube import Hypercube
from sep_python.sep_vector import FloatVector
io=sep_python.modes.default_io
vec=io.vector_from_storage("seabin.HH")


In [ ]:
import copy
import numpy as np
from sep_python.sep_vector import FloatVector,get_sep_vector
class boxFilter(FloatVector):
    """Class for defining a filter on a box"""
    def __init__(self,sh,mask,zero_lag,vals=None,space_only=False):
        """
        Create a filter that is a n-d cube with some of the fixed (probably zeroed
        
        sh   - N-d arry of shape
        zero_lag - Location of the zero lag in the filter
        
        mask - Mask for filter coefs n-d array (should be 0 or 1
        vals - Value for filter coeficients
        
        space_only - Whether or not actually have storage
        """
        if tuple(sh)!= mask.shape:
            raise Exception(f"Mask {mask.shape} and filter {sh} shape must be the same")
        
        if vals is not None:
            if vals.shape != sh:
                raise Exception("vals not the same shape")
                
        hyper=Hypercube.set_with_ns(sh)
        super().__init__(hyper,vals=vals)    
        self.mask=copy.deepcopy(mask)   
        self.zero=np.asarray(zero_lag)
        
    @classmethod
    def PEF(self,sh,one_loc):
        if len(sh) != len(one_loc):
            raise Exception("One location is not the same size as box filter shape")
        b=1
        one_1d=0
        for filt_s, one_s in zip(sh,one_loc):
            if one_s < 0 or one_s >= filt_s:
                raise Exception("Illegal locaiton for one in the filter")
            one_1d+one_s*b
            b*=filt_s
            
        msk=np.ones(sh,dtype=np.float32)
        m=np.ravel(msk)
        vals=np.zeros(sh,dtype=np.float32)
        v=np.ravel(vals)
        v[:one_1d]=0
        m[:one_1d+1]=0
        v[one_1d]=1
        return boxFilter(sh,msk,one_loc,vals=vals)

                     
    def clone(self):
        """Return a clone of the helix filter"""
        x=boxFilter(self.get_nd_array().shape,self.mask,self.zero,vals=self.get_nd_array())
        return x
    
    def cloneSpace(self):
        x=boxFilter(self.get_nd_array().shape,self.mask,self.zero,space_only=True)
        return x

    

In [ ]:
import sep_python.modes
import numpy as np
from sep_python.hypercube import Hypercube
from sep_python.sep_vector import FloatVector
io=sep_python.modes.default_io
vec=io.vector_from_storage("./seabin.HH")

In [ ]:
import numba
from genericSolver.pyOperator import Operator
class convOpAdjData(Operator):
    def __init__(self,model,data,filt):
        if not model.checkSame(data) and not isinstance(model,FloatVector):
            raise Exception("Expecting model and data to be the same shape and float vectors")
        self.filt=filt.clone()
        
        if len(self.filt.get_nd_array().shape) >len(model.get_nd_array().shape):
            raise Exception("Filter must be same number of dimensions or smaller than model/data")
        super().__init__(model,data)
        
        
    
    def forward(self,add,model,data):
        self.checkDomainRange(model,data)
        if not add:
            data.zero()
        m=model.get_nd_array()
        d=data.get_nd_array()
        f=self.filt.get_nd_array()
        print(type(m),type(f),type(d),"types")
        if len(m.shape)==2:
            if len(f.shape)==2:
                forward_2_2(m,f,d,self.filt.zero)
            else:
                forward_2_1(m,f,d,self.filt.zero)
        else:
            forward_1_1(m,f,d,self.filt.zero)
        
        
    
    def adjoint(self,add,model,data):
        self.checkDomainRange(model,data)
        if not add:
            model.zero()
        m=model.get_nd_array()
        d=data.get_nd_array() 
        f=self.filt.get_nd_array()
        

        if len(m.shape)==2:
            if len(f.shape)==2:
                adjoint_m_2_2(m,f,d,self.filt.zero)
            else:
                adjoint_m_2_1(m,f,d,self.filt.zero)
        else:
            adjoint_m_1_1(m,f,d,self.filt.zero)
        
class convOpAdjFilter(Operator):
    def __init__(self,filt,data,model):
        if not model.checkSame(data) and not isinstance(model,FloatVector):
            raise Exception("Expecting model and data to be the same shape and float vectors")
        self._model=model
        
        if len(self.filt.get_nd_array().shape) >len(model.get_nd_array().shape):
            raise Exception("Filter must be same number of dimensions or smaller than model/data")
        super().__init__(filt,data)
        
        
    
    def forward(self,add,filt,data):
        self.checkDomainRange(filt,data)
        if not add:
            data.zero()
        m=self.model.get_nd_array()
        d=data.get_nd_array()
        f=filt.get_nd_array()
        if len(m.shape)==2:
            if len(f.shape)==2: pass
               # forward_2_2(m,f,d,filt.zero)
            else:
                forward_2_1(m,f,d,filt.zero)
        else:
            forward_1_1(m,f,d,filt.zero)
        
        
    
    def adjoint(self,add,filt,data):
        self.checkDomainRange(filt,data)
        if not add:
            filt.zero()
        m=self.model.get_nd_array()
        d=data.get_nd_array() 
        ftemp=filter.clone()
        f=filt.get_nd_array()
        
        ft=ftemp.clone()
        ft.zero()

        if len(m.shape)==2:
            if len(f.shape)==2:
                adjoint_f_2_2(m,ft,d,filt.zero)
            else:
                adjoint_f_2_1(m,ft,d,filt.zero)
        else:
            adjoint_f_1_1(m,ft,d,filt.zero)
        ft[:]=ft[:]*filt.mask[:]
                
@numba.njit()
def forward_2_2(m ,f,d,zero):   
    return
    for i2 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
        for i1 in range(f.shape[1]-zero[1]-1,m.shape[1]-zero[1]):
            for if2 in range(0,f.shape[0]):
                for if1 in range(0,f.shape[1]):
                    d[i2,i1]+=f[if2,if1]*m[i2-if2+zero[0] ,i1-if1+zero[1]]

    
@numba.njit()
def forward_2_1(m,f,d,zero):               
    for i2 in range(m.shape):
        for i1 in range(f.shape[0]-zero[0]-1,m.shape[1]-zero[0]):
                for if1 in range(0,f.shape[1]):
                    d[i2,i1]+=f[if1]*m[i2 ,i1-if1+zero[0]]
                                 
@numba.njit()
def forward_1_1(m,f,d,zero):             
    for i1 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
            for if1 in range(0,f.shape[1]):
                d[i1]+=f[if1]*m[i1-if1+zero[0]]

@numba.njit()
def adjoint_m_2_2(m,f,d,zero):          
    for i2 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
        for i1 in range(f.shape[1]-zero[1]-1,m.shape[1]-zero[1]):
            for if2 in range(0,f.shape[0]):
                for if1 in range(0,f.shape[1]):
                    m[i2-if2+zero[0] ,i1-if1+zero[1]]+=d[i2,i1]*f[if2,if1]

@numba.njit()
def adjoint_m_2_1(m,f,d,zero):  
    for i2 in range(m.shape[0]):
        for i1 in range(f.shape[0]-zero[0]-1,m.shape[1]-zero[0]):
                for if1 in range(0,f.shape[0]):
                    m[i2,i1-if1+zero[1]]+=d[i2,i1]*f[if1]
  
@numba.njit()
def adjoint_m_1_1(m,f,d,zero):
    for i1 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
            for if1 in range(0,f.shape[0]):
                m[i1-if1+zero[1]]+=d[i1]*f[if1]
                
@numba.njit()
def adjoint_f_2_2(m,f,d,zero):          
    for i2 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
        for i1 in range(f.shape[1]-zero[1]-1,m.shape[1]-zero[1]):
            for if2 in range(0,f.shape[0]):
                for if1 in range(0,f.shape[1]):
                    f[if2,if1]+=m[i2-if2+zero[0] ,i1-if1+zero[1]]*d[i2,i1]

@numba.njit()
def adjoint_f_2_1(m,f,d,zero):  
    for i2 in range(m.shape[0]):
        for i1 in range(f.shape[0]-zero[0]-1,m.shape[1]-zero[0]):
                for if1 in range(0,f.shape[0]):
                    f[if1]+=m[i2,i1-if1+zero[1]]*d[i2,i1]
  
@numba.njit()
def adjoint_f_1_1(m,f,d,zero):
    for i1 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
            for if1 in range(0,f.shape[0]):              
                f[if1]+=m[i1-if1+zero[1]]*d[i1]

In [ ]:
import numba


import copy
class Jop(Operator):

  def __init__(self,mod:FloatVector,dat:FloatVector,sc):
    if not mod.checkSame(dat):
        raise Exception("Model and data must be the same")
    
    self._sc=copy.deepcopy(np.ravel(sc))
    if mod.get_hyper().get_n123()!=self._sc.shape[0]:
        raise Exception(f"Expecting mask {self.m.shape[0]} and model {mod.get_hyper().get_n123()} to be same size ")

    super().__init__(mod,dat)

  def forward(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      dat.zero()
    
    d=np.ravel(dat.get_nd_array())
    m=np.ravel(mod.get_nd_array())
    d[:]+=m[:]*self._sc[:]

  def adjoint(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      mod.zero()
    d=np.ravel(dat.get_nd_array())
    m=np.ravel(mod.get_nd_array())
    m[:]+=d[:]*self._sc[:]
  
class Laplacian(Operator):
    def __init__(self,model,data):
        super().__init__(model,data)
    def forward(self,add,model,data):
        if not add:
            data.scale(0.)
        lapjitF(model.getNdArray(),data.getNdArray())
    def adjoint(self,add,model,data):
        if not add:
            model.scale(0.)
        lapjitA(model.getNdArray(),data.getNdArray())

@numba.njit()
def lapjitF(model,data):
    for i2 in range(1,model.shape[0]-1):
        for i1 in range(1,model.shape[1]-1):
            data[i2,i1]=data[i2,i1]+model[i2,i1] -.25*(model[i2,i1-1]+model[i2,i1+1]+model[i2+1,i1]+model[i2-1,i1])

@numba.njit()
def lapjitA(model,data):
    for i2 in range(1,model.shape[0]-1):
        for i1 in range(1,model.shape[1]-1):
            model[i2,i1]=model[i2,i1]+data[i2,i1]
            model[i2,i1-1]=model[i2,i1-1]-.25*data[i2,i1]
            model[i2,i1+1]=model[i2,i1+1]-.25*data[i2,i1]
            model[i2-1,i1]=model[i2-1,i1]-.25*data[i2,i1]
            model[i2+1,i1]=model[i2+1,i1]-.25*data[i2,i1]



In [ ]:
from sep_plot import Grey

known=vec.clone()
@numba.njit()
def known_data(vec,known):
    for i2 in range(vec.shape[0]):
        for i1 in range(vec.shape[1]):
            if vec[i2,i1]==0:
                known[i2,i1]=0
            else:
                known[i2,i1]=1
known_data(vec.get_nd_array(),known.get_nd_array())

Grey(known)

In [ ]:
from genericSolver.pyProblem import ProblemL2Linear,ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 

j_op=Jop(vec,vec,known.get_nd_array())
problemStop=BasicStopper(niter=1000)
data=vec.clone()
model=data.clone()
reg=Laplacian(model,data)
reg.dotTest()
eps=.01

problemLap1=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg)
solve_base=LCGsolver(problemStop)
#solve_base.setDefaults(save_model=True)
solve_base.run(problemLap1)

Grey(problemLap1.model,pclip=99)+Grey(vec)



In [ ]:
from sep_plot import Grey
from sep_python.hypercube import Hypercube
@numba.njit()
def covFunc(model,nm1,nm2,cov):
    for i2 in range(model.shape[0]):
        for i1 in range(model.shape[1]):
            for im2 in range(-nm2,nm2):
                if i2+im2 >=0 and i2+im2 < model.shape[0]:
                    for im1 in range(-nm1,nm1):
                        if i1+im1 >=0 and i1+im1 < model.shape[1]:
                            cov[im2+nm2,im1+nm1]+=(model[i2,i1]*model[i2+im2,i1+im1])
cov=get_sep_vector(ns=[81,81],os=[-40,-40],ds=[1,1])
hyper=Hypercube.set_with_ns([81,81],os=[-40,-40],ds=[1,1])
covFunc(vec.getNdArray(),40,40,cov.get_nd_array())
Grey(cov,pclip=100)

In [ ]:
from genericSolver.pyProblem import ProblemL2Linear,ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 
from genericSolver.pyOperator import ChainOperator

msk=np.ones((41,41))
filt=boxFilter((41,41),msk,[21,21],cov.get_nd_array()[20:-20,20:-20])
prec=convOpAdjData(model,data,filt)
op=ChainOperator(prec,j_op)
eps=.001

problemPrec=ProblemL2LinearReg(model,data,op,eps,reg_op=reg)
solvePrec=LCGsolver(problemStop)
#solve_base.setDefaults(save_model=True)
solvePrec.run(problemPrec)

Grey(problemLap1.model,pclip=99)+Grey(vec)
#j_op=Jop(vec,vec,known.get_nd_array())
#problemStop=BasicStopper(niter=1000)
#data=vec.clone()
#model=data.clone()
#reg=Laplacian(model,data)
#reg.dotTest()
#eps=.01

#problemLap1=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg)
#solve_base=LCGsolver(problemStop)
#solve_base.setDefaults(save_model=True)
#solve_base.run(problemLap1)

#Grey(problemLap1.model,pclip=99)+Grey(vec)
